In [1]:
import cubewalkers as cw
import cupy as cp
from copy import deepcopy

In [2]:
IMPORT_RULES_FROM_FILES = True
cc_models_dir = './models/cell_collective/'
if IMPORT_RULES_FROM_FILES:
    from os import listdir
    
    sync_models = {}
    for fname in listdir(cc_models_dir):
        with open(cc_models_dir+fname) as rulefile:
            name = fname.strip('.txt')
            rules = rulefile.read()
            sync_models[name]=cw.Model(rules)
else:
    from cana.datasets.bio import load_all_cell_collective_models
    def cell_collective_models():
        return {BN.name:cw.Model(cw.conversions.network_rules_from_cana(BN)) 
                for BN in load_all_cell_collective_models()}
    sync_models = cell_collective_models()
    for name,model in sync_models.items():
        with open(cc_models_dir+name+'.txt','w') as rulefile:
            rulefile.write(model.rules)

total_models = len(sync_models)
async_models = deepcopy(sync_models)

In [9]:
difficult_models = {
    'Arabidopsis thaliana Cell Cycle': (5000,5000),
    'Guard Cell Abscisic Acid Signaling': (5000, 5000),
    'Signal Transduction in Fibroblasts': (20000, 50000),
}

def simulate_models(models, sync=True ,W=2500):
    convergence_measures = {}
    convergence_measures_alt = {}
    for model_idx, (model_name, model) in enumerate(sorted(models.items())):
        model.n_walkers = W
        N = model.n_variables
        
        if model_name in difficult_models:
            timescale, T_burn = difficult_models[model_name]
        else:
            timescale = N + 1000
            T_burn = N*50 + 1000
        
        T_window = 5 * timescale        
        T = T_burn + T_window            
        
        model.n_time_steps = T

        print(f"Simulating Model {model_name} ({W=},{T=},{T_burn=},{T_window=},{N=}). . .")
        if sync:
            model.simulate_ensemble(T_window=T_window,
                                    averages_only=True,
                                    maskfunction=cw.update_schemes.synchronous,
                                    threads_per_block=(16, 16))
        else:
            model.simulate_ensemble(T_window=T_window,
                                    averages_only=True,
                                    maskfunction=cw.update_schemes.asynchronous,
                                    threads_per_block=(16, 16))
            
        tw1 = model.trajectories[0*timescale:2*timescale]
        tw2 = model.trajectories[1*timescale:3*timescale]
        tw3 = model.trajectories[2*timescale:4*timescale]
        tw4 = model.trajectories[3*timescale:]
        convergence_measures[model_name] = max([
            cp.max(cp.abs(tw1.mean(axis=0) - tw2.mean(axis=0))),    
            cp.max(cp.abs(tw1.mean(axis=0) - tw3.mean(axis=0))),
            cp.max(cp.abs(tw1.mean(axis=0) - tw4.mean(axis=0))),
            cp.max(cp.abs(tw2.mean(axis=0) - tw3.mean(axis=0))),
            cp.max(cp.abs(tw2.mean(axis=0) - tw4.mean(axis=0))),
            cp.max(cp.abs(tw3.mean(axis=0) - tw4.mean(axis=0))),
            ])
        convergence_measures_alt[model_name] = max([
            cp.sum(cp.abs(tw1.mean(axis=0) - tw2.mean(axis=0))),    
            cp.sum(cp.abs(tw1.mean(axis=0) - tw3.mean(axis=0))),
            cp.sum(cp.abs(tw1.mean(axis=0) - tw4.mean(axis=0))),
            cp.sum(cp.abs(tw2.mean(axis=0) - tw3.mean(axis=0))),
            cp.sum(cp.abs(tw2.mean(axis=0) - tw4.mean(axis=0))),
            cp.sum(cp.abs(tw3.mean(axis=0) - tw4.mean(axis=0))),
            ])
        
        print(f"Progress: {(model_idx+1)}/{total_models},\t maximum difference: {convergence_measures[model_name]},\t sum difference: {convergence_measures_alt[model_name]}")
    return convergence_measures,convergence_measures_alt

In [10]:
sync_convergence_measures,sync_alt=simulate_models(sync_models,sync=True)

Simulating Model Apoptosis Network (W=2500,T=8255,T_burn=3050,T_window=5205,N=41). . .
Progress: 1/74,	 maximum difference: 8.914505283380558e-05,	 sum difference: 0.0006147934678189942
Simulating Model Arabidopsis thaliana Cell Cycle (W=2500,T=30000,T_burn=5000,T_window=25000,N=14). . .
Progress: 2/74,	 maximum difference: 6.84800000000374e-05,	 sum difference: 0.0005577599999996852
Simulating Model Aurora Kinase A in Neuroblastoma (W=2500,T=7265,T_burn=2150,T_window=5115,N=23). . .
Progress: 3/74,	 maximum difference: 0.0,	 sum difference: 0.0
Simulating Model B bronchiseptica and T retortaeformis coinfection (W=2500,T=8915,T_burn=3650,T_window=5265,N=53). . .
Progress: 4/74,	 maximum difference: 0.0,	 sum difference: 0.0
Simulating Model B cell differentiation (W=2500,T=7210,T_burn=2100,T_window=5110,N=22). . .
Progress: 5/74,	 maximum difference: 0.0,	 sum difference: 0.0
Simulating Model BT474 Breast Cell Line Long-term ErbB Network (W=2500,T=7375,T_burn=2250,T_window=5125,N=25). 

In [11]:
async_convergence_measures,async_alt=simulate_models(async_models,sync=False)

Simulating Model Apoptosis Network (W=2500,T=8255,T_burn=3050,T_window=5205,N=41). . .
Progress: 1/74,	 maximum difference: 0.005071085494716521,	 sum difference: 0.031153121998078792
Simulating Model Arabidopsis thaliana Cell Cycle (W=2500,T=30000,T_burn=5000,T_window=25000,N=14). . .
Progress: 2/74,	 maximum difference: 0.0006698399999998661,	 sum difference: 0.004383360000000447
Simulating Model Aurora Kinase A in Neuroblastoma (W=2500,T=7265,T_burn=2150,T_window=5115,N=23). . .
Progress: 3/74,	 maximum difference: 0.0028619745845551714,	 sum difference: 0.015488367546432014
Simulating Model B bronchiseptica and T retortaeformis coinfection (W=2500,T=8915,T_burn=3650,T_window=5265,N=53). . .
Progress: 4/74,	 maximum difference: 0.0,	 sum difference: 0.0
Simulating Model B cell differentiation (W=2500,T=7210,T_burn=2100,T_window=5110,N=22). . .
Progress: 5/74,	 maximum difference: 0.0,	 sum difference: 0.0
Simulating Model BT474 Breast Cell Line Long-term ErbB Network (W=2500,T=7375,

In [12]:
print(f'max max sync: {max(sync_convergence_measures.values())}')
print(f'max sum sycn: {max(sync_alt.values())}')
print(f'max max async: {max(async_convergence_measures.values())}')
print(f'max sum async: {max(async_alt.values())}')

max max sync: 0.0004160396039604164
max sum sycn: 0.003857563850687362
max max async: 0.006569114307344082
max sum async: 0.08810704999999806


In [13]:
with open('./data/converged_average_node_values.csv','w') as f:
    for model_name, smodel, amodel in [(k,sync_models[k],async_models[k]) for k in sorted(async_models)]:
        straj = ','.join(map(lambda x: str(cp.round(x,3)),cp.mean(smodel.trajectories,axis=0)))
        atraj = ','.join(map(lambda x: str(cp.round(x,3)),cp.mean(amodel.trajectories,axis=0)))
        f.write(f'{model_name},{cp.round(sync_convergence_measures[model_name],3)},{cp.round(sync_alt[model_name],3)},synchronous,{straj}\n')
        f.write(f'{model_name},{cp.round(async_convergence_measures[model_name],3)},{cp.round(async_alt[model_name],3)},asynchronous,{atraj}\n')
    